In [3]:
import compas_rrc as rrc
from compas.geometry import Point, Vector, Frame


# Start ROS & ABB clients
ros = rrc.RosClient()
ros.run()

abb = rrc.AbbClient(ros, '/rob1')
print('Connected to ABB IRB 6700')
    


Connected to ABB IRB 6700


In [4]:
done = abb.send_and_wait(rrc.SetTool('fingergrab'))
frame = abb.send_and_wait(GetFrame())
print("Active TCP frame:", frame)


NameError: name 'GetFrame' is not defined

In [ ]:
def open_gripper():
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO8', 0))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO6', 1))
    print(" → Gripper opened")

def close_gripper():
    abb.send_and_wait(rrc.WaitTime(2.0))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO6', 0))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO8', 1))
    # wait 2 seconds on the robot controller
    abb.send_and_wait(rrc.WaitTime(2.0))
    print(" → Gripper closed")

# Define frames with matching keys
frames = {
    "home":           Frame(Point(690.435653, -1306.975386, 1366.823567), Vector(0,1,0), Vector(1,0,0)),
    "safe_plane":     Frame(Point(196.3629, -1990.51,  1560.9089), Vector(0,1,0), Vector(1,0,0)),
    "approach_pick":  Frame(Point(-303.6371,-1990.51,  1560.9089), Vector(0,1,0), Vector(1,0,0)),
    "pick":           Frame(Point(-303.6371,-1990.51,  1260.9089), Vector(0,1,0), Vector(1,0,0)),
    "approach_place": Frame(Point(946.3629, -1990.51,  1560.9089), Vector(0,1,0), Vector(1,0,0)),
    "place":          Frame(Point(946.3629, -1990.51,  1260.9089), Vector(0,1,0), Vector(1,0,0)),}

speed_fly    = 2000  # mm/s for transit moves
speed_action =  200  # mm/s for pick/place
zone         = rrc.Zone.FINE

def do_move(step_name, frame, speed, motion):
    abb.send_and_wait(rrc.MoveToFrame(frame, speed, zone, motion))
    print(f" → Step '{step_name}' ({motion}) completed")


In [ ]:
do_move("Hitting home",    frames["home"], speed_action,    rrc.Motion.JOINT)

In [ ]:
open_gripper()
abb.send_and_wait(rrc.WaitTime(2.0))
close_gripper()

In [ ]:
# Sequential workflow
# do_move("Hitting home",    frames["home"],           speed_fly,    rrc.Motion.JOINT)
do_move("Safe plane",      frames["safe_plane"],     speed_fly,    rrc.Motion.JOINT)
do_move("Approach pick",   frames["approach_pick"],  speed_fly, rrc.Motion.JOINT)
open_gripper()
do_move("Pick",            frames["pick"],           speed_action, rrc.Motion.LINEAR)
close_gripper()
do_move("Retract pick",    frames["approach_pick"],  speed_action, rrc.Motion.LINEAR)
do_move("Approach place",  frames["approach_place"], speed_fly,    rrc.Motion.JOINT)
do_move("Place",           frames["place"],          speed_action, rrc.Motion.LINEAR)
abb.send_and_wait(rrc.WaitTime(2.0))
open_gripper()
abb.send_and_wait(rrc.WaitTime(2.0))
do_move("Retract place",   frames["approach_place"], speed_action, rrc.Motion.LINEAR)
do_move("Return to safe",  frames["safe_plane"],     speed_fly,    rrc.Motion.LINEAR)
do_move("Go home",         frames["home"],           speed_fly,    rrc.Motion.JOINT)

print("All steps completed. Shutting down.")
ros.close()
